In [197]:
import json
import random
import string
import time
import uuid

In [198]:
number_of_insurance_quotes = 100
insurance_quote_duplicates = 2

number_of_policies = 100
number_of_policy_updates = 5

customers_number = 10

# Insurance Quote Request and Policy Events (controlled)
Generates:
- 100 unique InsuranceQuoteRequest events
- 200 duplicate InsuranceQuoteRequest events (2 duplicates per insurance quote)
- 100 unique PolicyCreated events (one per insurance quote)
- 500 PolicyUpdated events (5 updates per policy). 


In [199]:
customer_ids = []
for i in range(customers_number):
    customer_ids += [''.join(random.choices(string.ascii_lowercase + string.digits, k=10))]

In [200]:
insurance_quote_ids = []
for i in range(number_of_insurance_quotes):
    insurance_quote_ids.append(i+1)

In [201]:
with open('insurance-quote.json', 'r') as file:
    insurance_quote_json = json.loads(file.read())

with open('policy-created.json', 'r') as file:
    policy_created_json = json.loads(file.read())

with open('update.json', 'r') as file:
    policy_updated_json = json.loads(file.read())
    
def jsons_with_id(insurance_quote, policy_created, policy_updated, i):
    insurance_quote_request_id = insurance_quote_ids[i]
    new_insurance_quote_request = insurance_quote.copy()
    customer_id = customer_ids[random.randint(0, customers_number - 1)]

    ts = int(time.time())
    
    new_insurance_quote_request['insuranceQuoteRequestDto']['id'] = insurance_quote_request_id
    new_insurance_quote_request['insuranceQuoteRequestDto']['customerInfo']['customerId'] = customer_id
    new_insurance_quote_request['date'] = ts

    new_policy_created = policy_created.copy() 
    new_policy_created['insuranceQuoteRequestId'] = insurance_quote_request_id
    policy_id =  ''.join(random.choices(string.ascii_lowercase, k=10))
    new_policy_created['policyId'] = policy_id

    new_policy_updated = policy_updated.copy()
    new_policy_updated['policy']['policyId'] = policy_id
    new_policy_updated['customer']['customerId'] = customer_id
    new_policy_updated['date'] = ts

    return new_insurance_quote_request, new_policy_created, new_policy_updated

In [202]:
all_events = []
for i in range(number_of_insurance_quotes):
    insurance_quote, policy_created, policy_updated = jsons_with_id(insurance_quote_json, policy_created_json, policy_updated_json, i)
    insurance_quotes = [insurance_quote] + [insurance_quote] * insurance_quote_duplicates
    policy_created = [policy_created]
    policy_updates = [policy_updated] * number_of_policy_updates
    all_events += list(map(lambda x: json.dumps(x), insurance_quotes)) + list(map(lambda x: json.dumps(x), policy_created)) +  list(map(lambda x: json.dumps(x), policy_updates))
    
random.shuffle(all_events)

In [203]:
with open('insurance-quote-and-policy-events-controlled.txt', 'w') as file:
    for event in all_events:
        file.write(str(uuid.uuid4()))
        file.write(" ")
        event_json = json.loads(event)
        event_id = event_json['insuranceQuoteRequestId'] if 'insuranceQuoteRequestId' in event_json else event_json['insuranceQuoteRequestDto']['id'] if 'insuranceQuoteRequestDto' in event_json else event_json['policy']['policyId']
        file.write(str(event_id))
        file.write(" ")
        file.write(event)
        file.write('\n')

In [204]:
all_events[0]

'{"date": 1660425972117, "insuranceQuoteRequestId": 57, "policyId": "ypjxoiyzed", "$type": "PolicyCreatedEvent"}'